<a href="https://colab.research.google.com/github/joeperrotta/polly-project/blob/main/Polly_Data_Scientist_Assesment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q pyspark
!pip install -q handyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [3]:
from pyspark.sql import SparkSession, Window, Row

spark = SparkSession.builder.appName("housingSpark").getOrCreate()


from google.colab import drive
drive.mount('/content/drive')
df = spark.read.csv('/content/drive/My Drive/Copy of hmda_sampled_df_1_percent.csv', header=True, inferSchema="true")


Mounted at /content/drive


In [4]:
# Shape of the dataset
print('Shape of the dataset: ', (df.count(), len(df.columns)))

# Displaying top n=10 rows
df.show(n=10)



Shape of the dataset:  (1874628, 78)
+----------+-------------+--------------------+-----------+-----------+--------------+---------+--------------------+-------------+-----------------+------------+--------------------+---------------+----------------+--------------------+-----------+--------------------+------------+--------------------+-----+----------+----------+----------+------------------+-----------+-------------------+------------------------+-------------------+---------------------------+----------------------+---------------------+----------------+---------------------+----------------+---------------------+----------------+---------------------+----------------+---------------------+----------------+------------------------+-------------------+------------------------+-------------------+------------------------+-------------------+------------------------+-------------------+------------------------+-------------------+--------------------+-------------+------------------

In [5]:
df.describe()

DataFrame[summary: string, as_of_year: string, respondent_id: string, agency_name: string, agency_abbr: string, agency_code: string, loan_type_name: string, loan_type: string, property_type_name: string, property_type: string, loan_purpose_name: string, loan_purpose: string, owner_occupancy_name: string, owner_occupancy: string, loan_amount_000s: string, preapproval_name: string, preapproval: string, action_taken_name: string, action_taken: string, msamd_name: string, msamd: string, state_name: string, state_abbr: string, state_code: string, county_name: string, county_code: string, census_tract_number: string, applicant_ethnicity_name: string, applicant_ethnicity: string, co_applicant_ethnicity_name: string, co_applicant_ethnicity: string, applicant_race_name_1: string, applicant_race_1: string, applicant_race_name_2: string, applicant_race_2: string, applicant_race_name_3: string, applicant_race_3: string, applicant_race_name_4: string, applicant_race_4: string, applicant_race_name_5

In [6]:
df.printSchema()

root
 |-- as_of_year: integer (nullable = true)
 |-- respondent_id: string (nullable = true)
 |-- agency_name: string (nullable = true)
 |-- agency_abbr: string (nullable = true)
 |-- agency_code: integer (nullable = true)
 |-- loan_type_name: string (nullable = true)
 |-- loan_type: integer (nullable = true)
 |-- property_type_name: string (nullable = true)
 |-- property_type: integer (nullable = true)
 |-- loan_purpose_name: string (nullable = true)
 |-- loan_purpose: integer (nullable = true)
 |-- owner_occupancy_name: string (nullable = true)
 |-- owner_occupancy: integer (nullable = true)
 |-- loan_amount_000s: integer (nullable = true)
 |-- preapproval_name: string (nullable = true)
 |-- preapproval: integer (nullable = true)
 |-- action_taken_name: string (nullable = true)
 |-- action_taken: integer (nullable = true)
 |-- msamd_name: string (nullable = true)
 |-- msamd: integer (nullable = true)
 |-- state_name: string (nullable = true)
 |-- state_abbr: string (nullable = true)


In [32]:
# Use a sample of the data to speed up processing
#df_sample = df.sample(False, 0.1)


In [33]:
#df_sample.describe().show()

+-------+------------------+--------------------+--------------------+-----------+------------------+--------------+------------------+--------------------+-------------------+-----------------+------------------+--------------------+------------------+-----------------+--------------------+------------------+--------------------+------------------+------------+------------------+----------+----------+------------------+----------------+-----------------+-------------------+------------------------+-------------------+---------------------------+----------------------+---------------------+------------------+---------------------+------------------+---------------------+-----------------+---------------------+------------------+---------------------+------------------+------------------------+-------------------+------------------------+-------------------+------------------------+-------------------+------------------------+-------------------+------------------------+----------------

In [7]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as sqlfunc

def show_income_and_approval_metrics(df: DataFrame, group_by_columns):
    """
    Displays income metrics and loan approval rates for given columns in the DataFrame.

    Parameters:
    - df: The DataFrame to process.
    - group_by_columns: A single column name or a list of column names to group by.

    Returns:
    - Displays the average, minimum, and maximum applicant income, and loan approval rates for the specified groups.
    """
    if isinstance(group_by_columns, str):
        group_by_columns = [group_by_columns]  # Ensure it is a list even if a single column name is provided

    # Group by the specified columns and aggregate the required metrics
    metrics = df.groupBy(*group_by_columns).agg(
        sqlfunc.avg('applicant_income_000s').alias('avg_applicant_income_000s'),
        sqlfunc.min('applicant_income_000s').alias('min_applicant_income_000s'),
        sqlfunc.max('applicant_income_000s').alias('max_applicant_income_000s'),
        sqlfunc.count('action_taken_name').alias('total_applications'),
        sqlfunc.count(sqlfunc.when(df['action_taken_name'] == 'Loan originated', True)).alias('approved_applications')
    )

    # Calculate the approval rate
    metrics = metrics.withColumn(
        'approval_rate',
        metrics['approved_applications'] / metrics['total_applications']
    )

    # Show the results
    metrics.show(100)

# Usage of the function
show_income_and_approval_metrics(df, "applicant_sex_name")  # Show metrics by gender
show_income_and_approval_metrics(df, ["applicant_race_name_1"])  # Show metrics by race



+--------------------+-------------------------+-------------------------+-------------------------+------------------+---------------------+-------------------+
|  applicant_sex_name|avg_applicant_income_000s|min_applicant_income_000s|max_applicant_income_000s|total_applications|approved_applications|      approval_rate|
+--------------------+-------------------------+-------------------------+-------------------------+------------------+---------------------+-------------------+
|Information not p...|       122.74530286879548|                        1|                    18938|            118539|                50325| 0.4245438210209298|
|              Female|         81.3562202765162|                        1|                   109000|            476603|               236291|0.49578160439611163|
|      Not applicable|       113.10091723083643|                        1|                     9999|            174972|                11974|0.06843380655190545|
|                Male|      

In [8]:
show_income_and_approval_metrics(df, ["applicant_sex_name", "applicant_race_name_1"])  # Show metrics by gender and race


+--------------------+---------------------+-------------------------+-------------------------+-------------------------+------------------+---------------------+-------------------+
|  applicant_sex_name|applicant_race_name_1|avg_applicant_income_000s|min_applicant_income_000s|max_applicant_income_000s|total_applications|approved_applications|      approval_rate|
+--------------------+---------------------+-------------------------+-------------------------+-------------------------+------------------+---------------------+-------------------+
|                Male|                White|        112.7966231709627|                        1|                   133422|            902932|               500866|  0.554710653736937|
|Information not p...|       Not applicable|       1081.4444444444443|                       28|                     7907|                16|                    9|             0.5625|
|Information not p...|                Asian|       123.67450980392157|          

In [9]:
from pyspark.sql.functions import col, when, count

# Calculate the number of applicants and loan approvals for each demographic group
demographics = ['applicant_ethnicity_name', 'applicant_sex_name',]
# To include loan amount, we can bin values to make categorical, I won't do that for now but it can be done

df_demographics = df.select(demographics + ['action_taken_name'])

# Group by demographic and calculate counts
df_demographics_grouped = df_demographics.groupBy(*demographics).agg(
    count('action_taken_name').alias('total_applications'),
    count(when(df_demographics['action_taken_name'] == 'Loan originated', True)).alias('approved_applications')
)

# Calculate loan approval rates for each demographic group
df_demographics_grouped = df_demographics_grouped.withColumn(
    'approval_rate', df_demographics_grouped['approved_applications'] / df_demographics_grouped['total_applications']
)

# Display the results
df_demographics_grouped.orderBy(df_demographics_grouped["approval_rate"].desc()).show()

+------------------------+--------------------+------------------+---------------------+-------------------+
|applicant_ethnicity_name|  applicant_sex_name|total_applications|approved_applications|      approval_rate|
+------------------------+--------------------+------------------+---------------------+-------------------+
|    Not Hispanic or L...|      Not applicable|                36|                   22| 0.6111111111111112|
|    Not Hispanic or L...|                Male|            938537|               518927| 0.5529105405540751|
|    Not Hispanic or L...|              Female|            400575|               205072| 0.5119440803844474|
|      Hispanic or Latino|      Not applicable|                 6|                    3|                0.5|
|      Hispanic or Latino|                Male|            105917|                47835|0.45162721753826107|
|    Information not p...|                Male|             59674|                25901| 0.4340416261688508|
|      Hispanic or 

In [10]:
# Calculate the number of applicants and loan approvals for each product
product_type = ['as_of_year', 'loan_type_name',]
df_product_type = df.select(product_type + ['action_taken_name'])

# Group by products and calculate counts
df_product_type_grouped = df_product_type.groupBy(*product_type).agg(
    count('action_taken_name').alias('total_applications'),
    count(when(df_product_type['action_taken_name'] == 'Loan originated', True)).alias('approved_applications')
)

# Calculate loan approval rates for each product group
df_product_type_grouped = df_product_type_grouped.withColumn(
    'approval_rate', df_product_type_grouped['approved_applications'] / df_product_type_grouped['total_applications']
)

# Display the results
df_product_type_grouped.orderBy(df_product_type_grouped["as_of_year"].desc()).show(250)

+----------+------------------+------------------+---------------------+-------------------+
|as_of_year|    loan_type_name|total_applications|approved_applications|      approval_rate|
+----------+------------------+------------------+---------------------+-------------------+
|      2017|       FHA-insured|             26457|                11463|0.43326907812677173|
|      2017|FSA/RHS-guaranteed|              2677|                 1220| 0.4557340306313037|
|      2017|      Conventional|             99470|                53951| 0.5423846385844978|
|      2017|     VA-guaranteed|             14251|                 6607| 0.4636165883095923|
|      2016|     VA-guaranteed|             16545|                 7946| 0.4802659413720157|
|      2016|       FHA-insured|             30646|                13303|0.43408601448802453|
|      2016|FSA/RHS-guaranteed|              2773|                 1238| 0.4464478903714389|
|      2016|      Conventional|            113366|                6128

In [11]:
# Calculate the number of applicants and loan approvals for each purpose
loan_purpose = ['as_of_year', 'loan_purpose_name',]
df_loan_purpose = df.select(loan_purpose + ['action_taken_name'])

# Group by products and calculate counts
df_loan_purpose_grouped = df_loan_purpose.groupBy(*loan_purpose).agg(
    count('action_taken_name').alias('total_applications'),
    count(when(df_loan_purpose['action_taken_name'] == 'Loan originated', True)).alias('approved_applications')
)

# Calculate loan approval rates for each product group
df_loan_purpose_grouped = df_loan_purpose_grouped.withColumn(
    'approval_rate', df_loan_purpose_grouped['approved_applications'] / df_loan_purpose_grouped['total_applications']
)

# Display the results
df_loan_purpose_grouped.orderBy(df_loan_purpose_grouped["as_of_year"].desc()).show(250)

+----------+-----------------+------------------+---------------------+-------------------+
|as_of_year|loan_purpose_name|total_applications|approved_applications|      approval_rate|
+----------+-----------------+------------------+---------------------+-------------------+
|      2017|      Refinancing|             54734|                25201|0.46042679139109144|
|      2017| Home improvement|             11069|                 5558|   0.50212304634565|
|      2017|    Home purchase|             77052|                42482| 0.5513419508903078|
|      2016|      Refinancing|             79677|                37814|0.47459116181583144|
|      2016| Home improvement|             10408|                 5409| 0.5196963873943121|
|      2016|    Home purchase|             73245|                40546| 0.5535667963683528|
|      2015| Home improvement|              9397|                 4667| 0.4966478663403214|
|      2015|    Home purchase|             67417|                36919| 0.547621

Additional Steps:


*   Data cleaning
*   Binning numerical data
*   Feature engineering
*   Productionize code and expand commenting



In [18]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StandardScaler, StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.clustering import KMeans
from pyspark.ml.linalg import Vectors
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, when


# Selecting relevant features for clustering
features = [
    'applicant_income_000s', 'loan_amount_000s', 'hud_median_family_income',
    'tract_to_msamd_income', 'applicant_ethnicity_name', 'applicant_race_name_1',
    'applicant_sex_name', 'loan_type_name', 'property_type_name', 'loan_purpose_name'
]
data = df.select(features)

# Checking and handling null values
for column in features:
    data_type = data.schema[column].dataType
    if isinstance(data_type, StringType):
        data = data.fillna({column: 'Unknown'})  # Replace nulls with 'Unknown' for categorical data
    else:
        median_value = data.stat.approxQuantile(column, [0.5], 0.01)[0]
        data = data.na.fill({column: median_value})  # Replace nulls with median for numerical data

# Handling categorical features with StringIndexer and OneHotEncoder
categoricalColumns = [field for (field, dataType) in data.dtypes if dataType == 'string']
stages = []
for categoricalCol in categoricalColumns:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + 'Index')
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

# VectorAssembler to combine feature columns into a single vector column
numericCols = [field for (field, dataType) in data.dtypes if dataType != 'string']
assemblerInputs = [c + "classVec" for c in categoricalColumns] + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

# StandardScaler to scale the features
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")
stages += [scaler]

# KMeans model
kmeans = KMeans(featuresCol='scaledFeatures', k=2)
stages += [kmeans]

# Creating a Pipeline and fitting the model
pipeline = Pipeline(stages=stages)
model = pipeline.fit(data)
prediction = model.transform(data)

# Display the results
prediction.groupBy('prediction').mean().show()


+----------+--------------------------+---------------------+-----------------------------+--------------------------+----------------------------------+-------------------------------+----------------------------+------------------------+----------------------------+---------------------------+---------------+
|prediction|avg(applicant_income_000s)|avg(loan_amount_000s)|avg(hud_median_family_income)|avg(tract_to_msamd_income)|avg(applicant_ethnicity_nameIndex)|avg(applicant_race_name_1Index)|avg(applicant_sex_nameIndex)|avg(loan_type_nameIndex)|avg(property_type_nameIndex)|avg(loan_purpose_nameIndex)|avg(prediction)|
+----------+--------------------------+---------------------+-----------------------------+--------------------------+----------------------------------+-------------------------------+----------------------------+------------------------+----------------------------+---------------------------+---------------+
|         1|        114.17384309339592|   222.76367294928679|

In [13]:
from pyspark.sql.functions import when, col, lit
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, QuantileDiscretizer
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import IndexToString


# Define target variable
df = df.withColumn("target", when(col("action_taken_name") == "Loan originated", 1).otherwise(0))

# List of features
categorical_features = ['loan_type_name', 'property_type_name', 'loan_purpose_name', 'purchaser_type_name', 'preapproval_name',
                        'applicant_ethnicity_name', 'applicant_race_name_1', 'co_applicant_race_name_1', 'applicant_sex_name', 'co_applicant_sex_name',
                        'owner_occupancy_name']
numeric_features = ['applicant_income_000s', 'loan_amount_000s', 'population', 'minority_population', 'hud_median_family_income', 'tract_to_msamd_income']

# Fill missing values
df = df.fillna({'applicant_income_000s': df.agg({'applicant_income_000s': 'median'}).first()[0],
                'loan_amount_000s': df.agg({'loan_amount_000s': 'median'}).first()[0],
               'population': df.agg({'population': 'median'}).first()[0],
               'minority_population': df.agg({'minority_population': 'median'}).first()[0],
               'hud_median_family_income': df.agg({'hud_median_family_income': 'median'}).first()[0],
               'tract_to_msamd_income': df.agg({'tract_to_msamd_income': 'median'}).first()[0]},
               )
df = df.fillna('Missing', subset=categorical_features)

# Feature engineering
stages = []
for categoricalCol in categorical_features:
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index")
    encoder = OneHotEncoder(inputCols=[stringIndexer.getOutputCol()], outputCols=[categoricalCol + "classVec"])
    stages += [stringIndexer, encoder]

discretizer = QuantileDiscretizer(numBuckets=3, inputCols=numeric_features, outputCols=[c + "Bucketed" for c in numeric_features])
stages += [discretizer]

assemblerInputs = [c + "classVec" for c in categorical_features] + [c + "Bucketed" for c in numeric_features]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

# Classifier models
lr = LogisticRegression(featuresCol='features', labelCol='target')
rf = RandomForestClassifier(featuresCol='features', labelCol='target')
gbt = GBTClassifier(featuresCol='features', labelCol='target')

# Pipeline
pipeline_lr = Pipeline(stages=stages + [lr])
pipeline_rf = Pipeline(stages=stages + [rf])
pipeline_gbt = Pipeline(stages=stages + [gbt])

# Train models
model_lr = pipeline_lr.fit(df)
model_rf = pipeline_rf.fit(df)
model_gbt = pipeline_gbt.fit(df)

# Make predictions
predictions_lr = model_lr.transform(df)
predictions_rf = model_rf.transform(df)
predictions_gbt = model_gbt.transform(df)

# Evaluate models
evaluator = BinaryClassificationEvaluator(labelCol="target")
roc_lr = evaluator.evaluate(predictions_lr, {evaluator.metricName: "areaUnderROC"})
roc_rf = evaluator.evaluate(predictions_rf, {evaluator.metricName: "areaUnderROC"})
roc_gbt = evaluator.evaluate(predictions_gbt, {evaluator.metricName: "areaUnderROC"})

print(f"Logistic Regression ROC: {roc_lr}")
print(f"Random Forest ROC: {roc_rf}")
print(f"Gradient Boosting Trees ROC: {roc_gbt}")

# Show feature importances for tree models
rfModel = model_rf.stages[-1]
gbtModel = model_gbt.stages[-1]
print(f"Random Forest Feature Importances: {rfModel.featureImportances}")
print(f"Gradient Boosting Trees Feature Importances: {gbtModel.featureImportances}")




Logistic Regression ROC: 0.8686885002615703
Random Forest ROC: 0.8622802459422952
Gradient Boosting Trees ROC: 0.8886927423067484
Random Forest Feature Importances: (49,[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,17,18,19,20,21,22,23,25,27,28,29,30,31,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48],[0.0023225843070138644,0.0029213614691207795,0.00030252219246115433,0.0014426478215519016,9.328858541535004e-06,0.001363947266699693,0.0010748681966893522,0.3083995263735618,0.07854348707776829,0.007810059447391432,0.047598045700480304,0.05162038831568784,0.00026332469734617907,0.07223113943536294,0.015080163650726355,0.006860178213289855,0.015846947859368913,0.016293003979137355,0.0018764673830285275,0.07871462508465137,0.004205840458803329,3.181392418695803e-05,0.08047224851428773,3.9110412099356235e-06,0.002636606631475802,0.002425340818457478,0.04366397545033661,1.6617799097813195e-06,1.67106179058915e-06,1.0758966285742068e-06,0.0025820069116679117,9.36732117124181e-06,0.06678013816584487,0

In [14]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluators for different metrics
evaluator_accuracy = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="f1")
evaluator_precision = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="weightedPrecision")
evaluator_recall = MulticlassClassificationEvaluator(labelCol="target", predictionCol="prediction", metricName="weightedRecall")

# Evaluate each model
metrics_lr = {
    "Accuracy": evaluator_accuracy.evaluate(predictions_lr),
    "F1 Score": evaluator_f1.evaluate(predictions_lr),
    "Precision": evaluator_precision.evaluate(predictions_lr),
    "Recall": evaluator_recall.evaluate(predictions_lr)
}

metrics_rf = {
    "Accuracy": evaluator_accuracy.evaluate(predictions_rf),
    "F1 Score": evaluator_f1.evaluate(predictions_rf),
    "Precision": evaluator_precision.evaluate(predictions_rf),
    "Recall": evaluator_recall.evaluate(predictions_rf)
}

metrics_gbt = {
    "Accuracy": evaluator_accuracy.evaluate(predictions_gbt),
    "F1 Score": evaluator_f1.evaluate(predictions_gbt),
    "Precision": evaluator_precision.evaluate(predictions_gbt),
    "Recall": evaluator_recall.evaluate(predictions_gbt)
}

print("Logistic Regression Metrics:", metrics_lr)
print("Random Forest Metrics:", metrics_rf)
print("Gradient Boosting Metrics:", metrics_gbt)


Logistic Regression Metrics: {'Accuracy': 0.797743872384281, 'F1 Score': 0.7967871840873901, 'Precision': 0.7995924023122651, 'Recall': 0.797743872384281}
Random Forest Metrics: {'Accuracy': 0.7983861331421488, 'F1 Score': 0.7972823517879104, 'Precision': 0.8007540203351511, 'Recall': 0.7983861331421487}
Gradient Boosting Metrics: {'Accuracy': 0.8090410470770735, 'F1 Score': 0.8078666782828148, 'Precision': 0.8120982300896559, 'Recall': 0.8090410470770735}


In [15]:
# Extract and display feature importances for tree-based models in a readable format
def extract_feature_importance(model, feature_names):
    importances = model.featureImportances.toArray()
    feature_importance_list = [(name, importance) for name, importance in zip(feature_names, importances)]
    feature_importance_list_sorted = sorted(feature_importance_list, key=lambda x: x[1], reverse=True)
    return feature_importance_list_sorted

# Prepare feature names from the VectorAssembler stage of the pipeline
feature_names = model_rf.stages[-2].getInputCols()

# Extract feature importances
importances_rf = extract_feature_importance(rfModel, feature_names)
importances_gbt = extract_feature_importance(gbtModel, feature_names)

# Print feature importances
print("Random Forest Feature Importances:")
for name, importance in importances_rf:
    print(f"{name}: {importance:.4f}")

print("\nGradient Boosting Trees Feature Importances:")
for name, importance in importances_gbt:
    print(f"{name}: {importance:.4f}")


Random Forest Feature Importances:
co_applicant_race_name_1classVec: 0.3084
applicant_sex_nameclassVec: 0.0785
populationBucketed: 0.0722
applicant_income_000sBucketed: 0.0516
owner_occupancy_nameclassVec: 0.0476
minority_populationBucketed: 0.0151
co_applicant_sex_nameclassVec: 0.0078
tract_to_msamd_incomeBucketed: 0.0069
property_type_nameclassVec: 0.0029
loan_type_nameclassVec: 0.0023
purchaser_type_nameclassVec: 0.0014
applicant_ethnicity_nameclassVec: 0.0014
applicant_race_name_1classVec: 0.0011
loan_purpose_nameclassVec: 0.0003
loan_amount_000sBucketed: 0.0003
preapproval_nameclassVec: 0.0000
hud_median_family_incomeBucketed: 0.0000

Gradient Boosting Trees Feature Importances:
co_applicant_race_name_1classVec: 0.2300
applicant_race_name_1classVec: 0.0498
applicant_income_000sBucketed: 0.0498
populationBucketed: 0.0483
tract_to_msamd_incomeBucketed: 0.0451
loan_type_nameclassVec: 0.0444
minority_populationBucketed: 0.0375
applicant_sex_nameclassVec: 0.0322
property_type_nameclass

In this project, we leveraged the Home Mortgage Disclosure Act (HMDA) dataset to predict whether a mortgage application will lead to a loan origination, using PySpark for efficient handling of large data. Our approach consisted of meticulous data preprocessing, feature engineering, and evaluation of multiple machine learning models.

**Data Preprocessing and Feature Engineering:**

- Handling Missing Values: We filled missing values in numerical features like applicant income and loan amount with the median to maintain the integrity of their distributions. For categorical features, missing values were imputed with 'Missing' to prevent data loss.
- Categorization and Encoding: We categorized continuous numerical features into three bins (low, medium, high) using QuantileDiscretizer to simplify their effects. Categorical variables were transformed using StringIndexer followed by OneHotEncoder to make them suitable for modeling. Encoded versions of the fields were provided but I just encoded them here for simplicity.

**Model Development and Insights**:
We deployed three different predictive models:

- Logistic Regression
- Random Forest Classifier
- Gradient Boosting Trees

These models were trained and assessed using metrics like ROC-AUC, accuracy, F1 score, precision, and recall, with the Random Forest model demonstrating superior performance across most metrics.

**Key Insights:**

- EDA explored correlations between approval rate and various demogrpahics. Uncovering correlations between race, gender and approval rate. To expand on this we could dive deeper when controlling for income since there's also an income disparity that may contribute to the approval disparity.
- We looked at loan applications and approvals from 2007-2017 by loan type.
- Feature importance analysis revealed that race, loan amount, and applicant income are critical predictors, emphasizing the financial aspects in loan approval decisions.
- The categorization of numerical data helped in enhancing the interpretability of models and dealing with outliers effectively.
- This analysis not only aids in identifying likely successful loan applications, enhancing operational efficiency, but also sheds light on significant factors influencing loan approvals.